##  오픈 API 불러오기

In [1]:
import geopandas as gpd
import requests, xmltodict, json
from shapely.geometry import Point, Polygon, LineString

requests를 통해 url을 불러올 수 있음

In [2]:
import requests, xmltodict, json

guGun=[230,110,170,200,140]


In [3]:
import pandas as pd
#for문을 통해 여러 url에 존재하는 데이터 값을 하나의 데이터프레임으로 생성
df1 = pd.DataFrame()
for i in guGun:
    region = i
    key = "gk1%2F6IyZtAC43Jd%2BEAt9wDo37%2BWsF503HOBB6yCbX7RLYbiIf85mxCCUAJRJIPn9OF%2F3vnivE4%2F2bHzvsku1aA%3D%3D"
    url = "http://apis.data.go.kr/B552061/frequentzoneLg/getRestFrequentzoneLg?searchYearCd=2017&siDo=30&guGun={}&numOfRows=10&pageNo=1&ServiceKey={}".format(region,key)
    content = requests.get(url).content
    dict = xmltodict.parse(content)
    jsonString = json.dumps(dict['response']['body'],ensure_ascii = False)
    json0bj = json.loads(jsonString) #파일 저장
    df = pd.DataFrame(json0bj['items']['item'])
    df1 = pd.concat([df,df1])

In [198]:
df1.head()

,index,afos_fid,afos_id,bjd_cd,spot_cd,sido_sgg_nm,spot_nm,occrrnc_cnt,caslt_cnt,dth_dnv_cnt,se_dnv_cnt,sl_dnv_cnt,wnd_dnv_cnt,geom_json,lo_crd,la_crd,geometry
0,0,6468067,2018074,30140111,30140001,대전광역시 중구1,대전광역시 중구 부사동(보문오거리 인근),35,52,0,10,42,0,"{""type"":""Polygon"",""coordinates"":[[[127.4361945...",127.434847043058,36.313142618138,"POLYGON ((127.43619 36.31314, 127.43617 36.312..."
1,1,6468068,2018074,30140110,30140002,대전광역시 중구2,대전광역시 중구 대사동(성모오거리 인근),26,59,2,6,51,0,"{""type"":""Polygon"",""coordinates"":[[[127.4215408...",127.420193324164,36.323995025951,"POLYGON ((127.42154 36.32400, 127.42151 36.323..."
2,2,6468069,2018074,30140115,30140003,대전광역시 중구3,대전광역시 중구 유천동(서부네거리 인근),23,52,0,4,48,0,"{""type"":""Polygon"",""coordinates"":[[[127.3928917...",127.391544230583,36.316890212714,"POLYGON ((127.39289 36.31689, 127.39287 36.316..."
3,0,6468073,2018074,30200125,30200001,대전광역시 유성구1,대전광역시 유성구 신성동(봉명네거리 인근),40,62,0,12,49,1,"{""type"":""Polygon"",""coordinates"":[[[127.3433051...",127.341957637336,36.355335369533,"POLYGON ((127.34331 36.35534, 127.34328 36.355..."
4,1,6468074,2018074,30200139,30200002,대전광역시 유성구2,대전광역시 유성구 반석동(외삼네거리 인근),33,63,1,5,55,2,"{""type"":""Polygon"",""coordinates"":[[[127.3131849...",127.311837481316,36.395885807785,"POLYGON ((127.31318 36.39589, 127.31316 36.395..."


특수문자 지우고, 8자리씩 뽑아서 float으로 만들어주고 2개씩 묶어서 tuple 그리고 tuple을 묶는 하나의 tuple 생성

In [186]:
#데이터가 들어갈 변수 생성(GeoJson으로 만들기 위해서는 지리정보를 포함한 geometry값이 필요)
df1['geometry']= None

In [187]:
#오픈 API에서 문자열로 인식하는 지리정보값을 geometry변수로 만들기 위해
#문자열을 새롭게 변경(리스트 -> 튜플)하는 과정 중 []가 필요하지 않아
#모두 제거 후 ','를 기준으로 값을 찾음
for j in range(15):
    abc = df1['geom_json'][j][32:-1]
    abc = abc.replace("[","")
    abc = abc.replace("]","")
    abc = abc.replace(" ","")
    latlot_list = []
    for i in range(33):
        empty=[]
        find_lat = abc.find(',')
        lat = float(abc[0:find_lat])
        abc = abc[find_lat+1:]
        find_lot = abc.find(',')
        lot = float(abc[0:find_lot])
        abc=abc[find_lot+1:]    
        empty.append(lat)
        empty.append(lot)
        tuple_l = tuple(empty)
        latlot_list.append(tuple_l)
    df1['geometry'][j] = Polygon(tuple(latlot_list))




<ipython-input-187-c585af89301c>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['geometry'][j] = Polygon(tuple(latlot_list))


In [188]:
df1

,index,afos_fid,afos_id,bjd_cd,spot_cd,sido_sgg_nm,spot_nm,occrrnc_cnt,caslt_cnt,dth_dnv_cnt,se_dnv_cnt,sl_dnv_cnt,wnd_dnv_cnt,geom_json,lo_crd,la_crd,geometry
0,0,6468067,2018074,30140111,30140001,대전광역시 중구1,대전광역시 중구 부사동(보문오거리 인근),35,52,0,10,42,0,"{""type"":""Polygon"",""coordinates"":[[[127.4361945...",127.434847043058,36.313142618138,"POLYGON ((127.43619452 36.31314262, 127.436168..."
1,1,6468068,2018074,30140110,30140002,대전광역시 중구2,대전광역시 중구 대사동(성모오거리 인근),26,59,2,6,51,0,"{""type"":""Polygon"",""coordinates"":[[[127.4215408...",127.420193324164,36.323995025951,"POLYGON ((127.4215408 36.32399503, 127.4215149..."
2,2,6468069,2018074,30140115,30140003,대전광역시 중구3,대전광역시 중구 유천동(서부네거리 인근),23,52,0,4,48,0,"{""type"":""Polygon"",""coordinates"":[[[127.3928917...",127.391544230583,36.316890212714,"POLYGON ((127.3928917 36.31689021, 127.3928658..."
3,0,6468073,2018074,30200125,30200001,대전광역시 유성구1,대전광역시 유성구 신성동(봉명네거리 인근),40,62,0,12,49,1,"{""type"":""Polygon"",""coordinates"":[[[127.3433051...",127.341957637336,36.355335369533,"POLYGON ((127.34330511 36.35533537, 127.343279..."
4,1,6468074,2018074,30200139,30200002,대전광역시 유성구2,대전광역시 유성구 반석동(외삼네거리 인근),33,63,1,5,55,2,"{""type"":""Polygon"",""coordinates"":[[[127.3131849...",127.311837481316,36.395885807785,"POLYGON ((127.31318495 36.39588581, 127.313159..."
5,2,6468075,2018074,30200118,30200003,대전광역시 유성구3,대전광역시 유성구 갑동(한밭대입구삼거리 인근),29,52,0,14,37,1,"{""type"":""Polygon"",""coordinates"":[[[127.3054750...",127.304127605106,36.360485181647,"POLYGON ((127.30547508 36.36048518, 127.305449..."
6,0,6468070,2018074,30170112,30170001,대전광역시 서구1,대전광역시 서구 둔산동(계룡로네거리 인근),68,86,0,12,71,3,"{""type"":""Polygon"",""coordinates"":[[[127.3786942...",127.377346731468,36.349725057766,"POLYGON ((127.3786942 36.34972506, 127.3786683..."
7,1,6468071,2018074,30170111,30170002,대전광역시 서구2,대전광역시 서구 갈마동(갈마네거리 인근),57,85,0,11,71,3,"{""type"":""Polygon"",""coordinates"":[[[127.3702199...",127.368872451304,36.35308630539,"POLYGON ((127.37021992 36.35308631, 127.370194..."
8,2,6468072,2018074,30170112,30170003,대전광역시 서구3,대전광역시 서구 둔산동(은하수네거리 인근),43,53,0,6,44,3,"{""type"":""Polygon"",""coordinates"":[[[127.3797323...",127.378384917831,36.351338057506,"POLYGON ((127.37973239 36.35133806, 127.379706..."
9,0,6468064,2018074,30110115,30110001,대전광역시 동구1,대전광역시 동구 용전동(동부네거리 인근),35,69,0,7,62,0,"{""type"":""Polygon"",""coordinates"":[[[127.4434238...",127.44207635574,36.351081306458,"POLYGON ((127.44342383 36.35108131, 127.443397..."


In [4]:
df1.reset_index(inplace=True)

In [5]:
df1

,index,afos_fid,afos_id,bjd_cd,spot_cd,sido_sgg_nm,spot_nm,occrrnc_cnt,caslt_cnt,dth_dnv_cnt,se_dnv_cnt,sl_dnv_cnt,wnd_dnv_cnt,geom_json,lo_crd,la_crd
0,0,6468067,2018074,30140111,30140001,대전광역시 중구1,대전광역시 중구 부사동(보문오거리 인근),35,52,0,10,42,0,"{""type"":""Polygon"",""coordinates"":[[[127.4361945...",127.434847043058,36.313142618138
1,1,6468068,2018074,30140110,30140002,대전광역시 중구2,대전광역시 중구 대사동(성모오거리 인근),26,59,2,6,51,0,"{""type"":""Polygon"",""coordinates"":[[[127.4215408...",127.420193324164,36.323995025951
2,2,6468069,2018074,30140115,30140003,대전광역시 중구3,대전광역시 중구 유천동(서부네거리 인근),23,52,0,4,48,0,"{""type"":""Polygon"",""coordinates"":[[[127.3928917...",127.391544230583,36.316890212714
3,0,6468073,2018074,30200125,30200001,대전광역시 유성구1,대전광역시 유성구 신성동(봉명네거리 인근),40,62,0,12,49,1,"{""type"":""Polygon"",""coordinates"":[[[127.3433051...",127.341957637336,36.355335369533
4,1,6468074,2018074,30200139,30200002,대전광역시 유성구2,대전광역시 유성구 반석동(외삼네거리 인근),33,63,1,5,55,2,"{""type"":""Polygon"",""coordinates"":[[[127.3131849...",127.311837481316,36.395885807785
5,2,6468075,2018074,30200118,30200003,대전광역시 유성구3,대전광역시 유성구 갑동(한밭대입구삼거리 인근),29,52,0,14,37,1,"{""type"":""Polygon"",""coordinates"":[[[127.3054750...",127.304127605106,36.360485181647
6,0,6468070,2018074,30170112,30170001,대전광역시 서구1,대전광역시 서구 둔산동(계룡로네거리 인근),68,86,0,12,71,3,"{""type"":""Polygon"",""coordinates"":[[[127.3786942...",127.377346731468,36.349725057766
7,1,6468071,2018074,30170111,30170002,대전광역시 서구2,대전광역시 서구 갈마동(갈마네거리 인근),57,85,0,11,71,3,"{""type"":""Polygon"",""coordinates"":[[[127.3702199...",127.368872451304,36.35308630539
8,2,6468072,2018074,30170112,30170003,대전광역시 서구3,대전광역시 서구 둔산동(은하수네거리 인근),43,53,0,6,44,3,"{""type"":""Polygon"",""coordinates"":[[[127.3797323...",127.378384917831,36.351338057506
9,0,6468064,2018074,30110115,30110001,대전광역시 동구1,대전광역시 동구 용전동(동부네거리 인근),35,69,0,7,62,0,"{""type"":""Polygon"",""coordinates"":[[[127.4434238...",127.44207635574,36.351081306458


In [9]:
#csv 저장
df1.to_csv('C:/Users/ParkMJ/Desktop/accident.csv')

In [190]:
#변환 데이터 geojson으로 저장
df1 = gpd.GeoDataFrame(df1)#,geometry='geometry')
df1.to_file('C:/Users/ParkMJ/Desktop/df1.geojson', driver='GeoJSON')

In [192]:
with open('C:/Users/ParkMJ/Desktop/df1.geojson',encoding='utf-8') as f:
    abc = json.load(f)

In [196]:
import folium

In [197]:
#GeoJson 시각화
center = [36.35111,127.38500]
m = folium.Map(location = center, zoom_start = 12)
folium.GeoJson(abc).add_to(m)
m

## 새로운 오픈 API 불러오기

In [59]:
url = ["https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14176255.864725836%2C4359830.506502272%2C14176867.360952117%2C4360442.0027285535)",
      "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14168551.01320776%2C4355427.733206511%2C14169162.50943404%2C4356039.229432792)",
      "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14167694.917557897%2C4349007.023996892%2C14168306.413784178%2C4349618.520223174)",
      "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14170324.351797441%2C4343503.557493827%2C14170935.848023722%2C4344115.053720108)",
      "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14175888.966523532%2C4339773.429113907%2C14176500.462749815%2C4340384.925340188)",
       "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14180230.58879706%2C4338183.539858644%2C14180842.085023344%2C4338795.036084926)",
       "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14185122.558607312%2C4339712.280424347%2C14185734.054833595%2C4340323.776650629)",
       "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14186651.300572619%2C4333475.01868301%2C14187262.796798902%2C4334086.514909292)",
       "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14189647.633014468%2C4330967.883921989%2C14190259.12924075%2C4331579.38014827)",
       "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14192093.61558692%2C4338244.689947807%2C14192705.111813203%2C4338856.186174088)",
       "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14189036.138187787%2C4348640.1253280565%2C14189647.63441407%2C4349251.621554337)",
       "https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14185611.757921008%2C4356773.025837401%2C14186223.25414729%2C4357384.522063683)"]

In [60]:
url[0]

'https://api.vworld.kr/req/data?key=AB8006FF-875C-3A19-82CC-E49E01D6EF3D&domain=http%3A%2F%2Flocalhost%3A8080&service=data&version=2.0&request=getfeature&format=json&size=100&page=1&data=LT_L_TRKROAD&geometry=true&attribute=true&crs=EPSG%3A3857&geomfilter=BOX(14176255.864725836%2C4359830.506502272%2C14176867.360952117%2C4360442.0027285535)'

In [61]:
import urllib.request
result_df = pd.DataFrame()
cat_nam_list = []
lnk_nam_list = []
cos_num_list = []
cos_nam_list = []
len_tim_list = []
geometry_list = []
for i in url:
    responseRoad = urllib.request.urlopen(i).read().decode('utf-8')
    jsonArray = json.loads(responseRoad)
    cat_nam_list.append(jsonArray['response']['result']['featureCollection']['features'][0]['properties']['cat_nam'])
    lnk_nam_list.append(jsonArray['response']['result']['featureCollection']['features'][0]['properties']['lnk_nam'])
    cos_num_list.append(jsonArray['response']['result']['featureCollection']['features'][0]['properties']['cos_num'])
    cos_nam_list.append(jsonArray['response']['result']['featureCollection']['features'][0]['properties']['cos_nam'])
    len_tim_list.append(jsonArray['response']['result']['featureCollection']['features'][0]['properties']['len_tim'])
    geometry_list.append(jsonArray['response']['result']['featureCollection']['features'][0]['geometry']['coordinates'][0])
result_df['cat_name'] = cat_nam_list
result_df['lnk_name']  = lnk_nam_list
result_df['cos_numb']  = cos_num_list
result_df['cos_name']  = cos_nam_list
result_df['len_time']  = len_tim_list
result_df['geometry_list']  = geometry_list

In [62]:
result_df['line'] = jsonArray['response']['result']['featureCollection']['features'][0]['geometry']['type']

In [63]:
import geopandas

In [64]:
result_df

,cat_name,lnk_name,cos_numb,cos_name,len_time,geometry_list,line
0,둘레길링크,금병산길,7코스,대전둘레길,←7.5시간 12.2km→,"[[14180377.105424806, 4362512.217485437], [141...",MultiLineString
1,둘레길링크,우산봉길,8코스,대전둘레길,←6시간 9km→,"[[14170941.844456036, 4357831.044832779], [141...",MultiLineString
2,둘레길링크,수통골길,9코스,대전둘레길,←6시간 10km→,"[[14169993.370674368, 4350287.438029166], [141...",MultiLineString
3,둘레길링크,성북동산성길,10코스,대전둘레길,←5.5시간 7.9km→,"[[14172479.166712204, 4340035.309312579], [141...",MultiLineString
4,둘레길링크,구봉산길,11코스,대전둘레길,←6.5시간 9.4km→,"[[14172479.166712204, 4340035.309312579], [141...",MultiLineString
5,둘레길링크,동물원길,12코스,대전둘레길,←7시간 11.5km→,"[[14179509.560050152, 4339984.445254406], [141...",MultiLineString
6,둘레길링크,보문산길,1코스,대전둘레길,←6시간 9.3km→,"[[14184523.427172402, 4343790.486646731], [141...",MultiLineString
7,둘레길링크,만인산길,2코스,대전둘레길,←7.5시간 13.1km→,"[[14185959.271508856, 4335620.8560022805], [14...",MultiLineString
8,둘레길링크,머들령길,3코스,대전둘레길,←7.5시간 12.5km→,"[[14187565.078457292, 4328173.036622399], [141...",MultiLineString
9,둘레길링크,식장산길,4코스,대전둘레길,←7.5시간 13.6km→,"[[14190903.351409525, 4336123.31095105], [1419...",MultiLineString


In [23]:
result_df.to_json('C:/Users/ParkMJ/Desktop/road.json')